<a href="https://colab.research.google.com/github/danielsoy/ALOCC-CVPR2018/blob/master/anyresnet1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [30]:
resnet2train=0  # cero=18. uno=50 00 01 10 11
     trained=0 # cero false, uno true.
epocas=50
####################################################################

In [13]:
if(trained==0):
	pretrained=False
	pre_train="_scratch"
else:
	pretrained=True
	pre_train="_preT"

In [14]:
if(resnet2train==1):
	model = models.resnet50(pretrained=trained)
	saved_model = models.resnet50(pretrained=trained)
	custom="50"

In [15]:
if(resnet2train==0):
	model = models.resnet18(pretrained=trained)
	saved_model = models.resnet18(pretrained=trained)
	custom="18"

Check if GPU is available

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


Custom Dataset Class

In [17]:
class SingleClassDataset(Dataset):
	def __init__(self, root_dir, transform=None):
		self.root_dir = root_dir
		self.transform = transform
		self.image_paths = [os.path.join(root_dir, img) for img in os.listdir(root_dir) if img.endswith(('.jpg', '.png', '.jpeg'))]

	def __len__(self):
		return len(self.image_paths)

	def __getitem__(self, idx):
		img_path = self.image_paths[idx]
		image = Image.open(img_path).convert('RGB')  # Ensure 3 channels
		label = 0  # Since there's only one class, all labels are 0

		if self.transform:
			image = self.transform(image)

		return image, label

Define transformations (no resizing needed, just normalization)

In [18]:
transform = transforms.Compose([
	transforms.ToTensor(),  # Convert images to PyTorch tensors
	transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

Load the dataset

In [24]:
dataset_path = r"/content/drive/MyDrive/dataset tres/normal"
dataset = SingleClassDataset(root_dir=dataset_path, transform=transform)

Split the dataset into training and validation sets

In [25]:
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

Create data loaders

In [26]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Load the pre-trained ResNet-18 model<br>
odel = models.resnet50(pretrained=True)

Modify the final layer for binary classification (1 output neuron)

In [27]:
model.fc = nn.Linear(model.fc.in_features, 1)

Move the model to the GPU if available

In [28]:
model = model.to(device)

Define loss function and optimizer

In [29]:
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss with logits
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training loop

In [31]:
num_epochs = epocas
for epoch in range(num_epochs):
	model.train()
	running_loss = 0.0
	for inputs, labels in train_loader:
		inputs, labels = inputs.to(device), labels.float().to(device)  # Labels must be float for BCEWithLogitsLoss

		optimizer.zero_grad()
		outputs = model(inputs).squeeze(1)  # Remove extra dimension
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()

		running_loss += loss.item()

	print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

	# Validation loop
	model.eval()
	val_loss = 0.0
	correct = 0
	total = 0
	with torch.no_grad():
		for inputs, labels in val_loader:
			inputs, labels = inputs.to(device), labels.float().to(device)
			outputs = model(inputs).squeeze(1)
			loss = criterion(outputs, labels)
			val_loss += loss.item()

			predicted = (torch.sigmoid(outputs) > 0.5).float()  # Convert logits to binary predictions
			total += labels.size(0)
			correct += (predicted == labels).sum().item()

	print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")

Epoch 1/1, Loss: 0.05785210761759016
Validation Loss: 0.0, Accuracy: 100.0%


Save the trained model

In [34]:
# Save the trained model
torch.save(model.state_dict(), "resnet" + custom + pre_train +"_custom.pth")

In [33]:
print("Model saved as resnet" + custom + pre_train + "_custom.pth")

Model saved as resnet18_scratch_custom.pth


In [40]:
#"/content/drive/MyDrive/dataset tres/normal"

torch.save(model.state_dict(), "/content/drive/MyDrive/dataset tres/" + "resnet" + custom + pre_train +"_custom.pth")




